In [64]:
import pandas as pd
import pyodbc

# Connexion à la base de données
server = 'DESKTOP-I9NMGNC'
database = 'DW_Finance'
username = 'amin'
password = 'amin'

conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)
conn = pyodbc.connect(conn_str)

# Requête SQL pour récupérer les données nécessaires
sql_query = """
SELECT 
    f.Fk_Produit, f.Fk_Invoices, f.Fk_Supplier, f.Fk_Date, f.Fk_Geographie, 
    f.Fk_InvoiceDate, f.Fk_DueDate, f.Fk_PaymentDate, f.Price, f.Amount, 
    f.DiscountOffered, f.RecommendedProfitMargin,
    d.PaymentDate, d.DueDate
FROM 
    DW_Finance.dbo.Fact_Purchase f
JOIN 
    DW_Finance.dbo.Dim_Supplier d ON f.Fk_Supplier = d.Pk_Supplier
"""

# Charger les données dans un DataFrame
df = pd.read_sql(sql_query, conn)

# Afficher les premières lignes du DataFrame
print(df.head())

# Fermer la connexion
conn.close()


   Fk_Produit  Fk_Invoices  Fk_Supplier  Fk_Date  Fk_Geographie  \
0         996         7256          243     8776             78   
1         760         3624          183    10456             18   
2         771         6001          189     8929             24   
3         871          552          219     9008             54   
4         981          638          240     9721             75   

   Fk_InvoiceDate  Fk_DueDate  Fk_PaymentDate  Price   Amount  \
0            8474        9291            8776    3.0  4133.94   
1            9166        8729           10456    2.5  3941.63   
2            8674        8775            8929    1.8  3081.99   
3            9628        9331            9008   22.0   820.28   
4            9230        9312            9721    7.0  1836.40   

   DiscountOffered  RecommendedProfitMargin PaymentDate     DueDate  
0           488.75                     45.0  2021-01-11  2022-06-10  
1            88.94                     40.0  2025-08-18  2020-11-2

C:\Users\Amin\AppData\Local\Temp\ipykernel_21104\2120349896.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


In [46]:
pip install sqlalchemy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [65]:
# Séparer les colonnes numériques et datetime
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
datetime_cols = df.select_dtypes(include=['datetime', 'datetime64[ns]']).columns

# Remplacer les valeurs manquantes pour les colonnes numériques par la moyenne
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())

# Remplacer les valeurs manquantes pour les colonnes datetime par la date la plus récente
for col in datetime_cols:
    df[col] = df[col].fillna(df[col].max())

# Vérifier les résultats
print(df.isnull().sum())


Fk_Produit                 0
Fk_Invoices                0
Fk_Supplier                0
Fk_Date                    0
Fk_Geographie              0
Fk_InvoiceDate             0
Fk_DueDate                 0
Fk_PaymentDate             0
Price                      0
Amount                     0
DiscountOffered            0
RecommendedProfitMargin    0
PaymentDate                0
DueDate                    0
dtype: int64


In [80]:
# Définir X et y pour l'entraînement du modèle
X = df[['Price', 'Amount', 'DiscountOffered']]  # Variables explicatives
y = df['RecommendedProfitMargin']  # Variable cible


In [81]:
from sklearn.model_selection import train_test_split

# Séparer les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [82]:
from sklearn.ensemble import RandomForestRegressor

# Créer le modèle Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entraîner le modèle avec les données d'entraînement
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [84]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prédire les valeurs sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluer les performances
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')


MAE: 1.1281025862068967
MSE: 4.108500334396552
R²: 0.7482915323866163


In [85]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Générer des données de test aléatoires
np.random.seed(42)
new_data = pd.DataFrame({
    'Price': np.random.randint(1000, 5000, size=5),  # Valeurs aléatoires pour 'Price' entre 1000 et 5000
    'Amount': np.random.randint(2000, 6000, size=5),  # Valeurs aléatoires pour 'Amount' entre 2000 et 6000
    'DiscountOffered': np.random.randint(5, 30, size=5),  # Valeurs aléatoires pour 'DiscountOffered' entre 5% et 30%
  
})

# Afficher les nouvelles données pour vérifier
print("Nouvelles données générées :")
print(new_data)

# Appliquer le scalage sur les colonnes numériques (ici, 'Price', 'Amount', 'DiscountOffered')
scaler = StandardScaler()
new_data[['Price', 'Amount', 'DiscountOffered']] = scaler.fit_transform(new_data[['Price', 'Amount', 'DiscountOffered']])

# Maintenant tu peux faire la prédiction
new_predictions = model.predict(new_data)

# Afficher les prédictions
print("Prédictions pour les nouvelles données : ", new_predictions)


Nouvelles données générées :
   Price  Amount  DiscountOffered
0   4174    3095               23
1   4507    5772               27
2   1860    5092               15
3   2294    3638               15
4   2130    4169               28
Prédictions pour les nouvelles données :  [39.7968 39.7968 35.1    35.1    35.1   ]


In [101]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Supposons que tu aies déjà préparé tes données dans df
X = df[['Price', 'Amount', 'DiscountOffered']]
y = df['RecommendedProfitMargin']

# Séparer les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer un modèle XGBoost pour la régression
model1 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6)

# Entraîner le modèle
model1.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = model1.predict(X_test)

# Évaluer les performances
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')


MAE: 0.7260115906287885
MSE: 2.3043588813797644
R²: 0.858822783094994


In [87]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Définir les paramètres à tester
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Créer le modèle
xgb = XGBRegressor(random_state=42)

# Configurer GridSearchCV
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Entraîner le modèle avec GridSearch
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print(f"Meilleurs paramètres: {grid_search.best_params_}")

# Prédictions avec les meilleurs paramètres
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Évaluer la performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')


Meilleurs paramètres: {'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 100, 'subsample': 1.0}
MAE: 0.6913944625854493
MSE: 1.9165789176818033
R²: 0.882580235325535


In [103]:
# Evaluate model on test data
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# Evaluate performance on training and test data
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

print(f"Train MAE: {train_mae}")
print(f"Test MAE: {test_mae}")

# Compare predictions
print("Predictions on test data:", y_pred_test[:5])  # Print the first 5 predictions
print("Actual values on test data:", y_test[:5])  # Print the first 5 actual values


Train MAE: 0.08417006306834035
Test MAE: 0.6913944625854493
Predictions on test data: [44.10293  45.350967 21.394766 39.045826 43.987705]
Actual values on test data: 257    45.00
574    45.00
485    20.79
101    45.00
566    45.00
Name: RecommendedProfitMargin, dtype: float64
